The purpose of this notebook is to take the initial model produced by model_train.ipynb and use the same functionality to train over more labelled records. 

You should be able to run this over and over again and see better and better results on the test dataset. 

In [ ]:
import pandas as pd
import torch 
import numpy as np
from data_helpers import feature_prep, split_sample
from model import load_ckp, BERTClass, train_model